### Reading exported data from slack

In [4]:

import json
import os
from pprint import pprint
path = '/home/bcardoen/Downloads/mial/MIAL Slack export Sep 28 2018 - Dec 13 2018' ## Unzip export.zip here

def treedir(pathname):
    (t, d, f)  = next(os.walk(pathname))
    return {**{dir:treedir(os.path.join(pathname, dir)) for dir in d}, **{file:os.path.join(t, file) for file in f}}

# with open('data.json') as f:
#     data = json.load(f)

# pprint(data)

In [10]:
contents = treedir(path)

In [12]:
print('\t Have channels:')
for c,v in contents.items():
    if not isinstance(v,str):
        print('\t {}'.format(c))

	 Have channels:
	 random
	 ai_news
	 smlm
	 general
	 ai_events
	 plant_image_analysis
	 derm
	 reading_groups
	 mial_infrastructure
	 cats
	 mial_social
	 recommended_papers
	 software_tricks_tools
	 compute_canada


In [17]:
dates_general = contents['general']
print('\t Have {} dates'.format(len(dates_general.keys())))

	 Have 44 dates


In [18]:
f = dates_general['2018-11-23.json']

In [19]:
with open(f) as file:
    data = json.load(file)

In [20]:
data

[{'latest_reply': '1543100057.165000',
  'replies': [{'ts': '1543100057.165000', 'user': 'UD3QNJE85'}],
  'reply_count': 1,
  'reply_users': ['UD3QNJE85'],
  'reply_users_count': 1,
  'text': '<https://twitter.com/tcia_news/status/1066075453260267522?s=12>',
  'thread_ts': '1543031015.164700',
  'ts': '1543031015.164700',
  'type': 'message',
  'user': 'UD562L1JS'},
 {'attachments': [{'fallback': 'Futurism: See the 3D images produced by the first full-body medical scanner',
    'from_url': 'https://futurism.com/medical-scanner-explorer/',
    'id': 1,
    'image_bytes': 40696,
    'image_height': 250,
    'image_url': 'https://wp-assets.futurism.com/2018/11/worlds-first-total-body-scanner-1200x630.jpg',
    'image_width': 476,
    'original_url': 'https://futurism.com/medical-scanner-explorer/',
    'service_icon': 'https://futurism.com/static/favicon.png',
    'service_name': 'Futurism',
    'text': "Even the device's creators were impressed by the clarity of the images.",
    'title'